In [185]:
import numpy as np

In [186]:
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))

## 1.1 ##

In [187]:
sigma = 0.30
a = 0.05
deltat = 1/12
V = sigma**2 * (1-np.exp(-2*a*deltat))/(2*a)
deltau = np.sqrt(3*V)
numMonths = 24
P = lambda x: np.e**(-0.06 * x * deltat)

In [188]:
OU = np.zeros((numMonths*2 + 1,numMonths))

In [189]:
for i in range(int(numMonths)):
    index1 = int(numMonths - i - 1)
    index2 = int(numMonths + i + 1)
    OU[index1][:] = (i+1) * deltau
    OU[index2][:] = -(i+1) * deltau

## 1.2 ##

In [190]:
intermediateParams = np.zeros((numMonths*2 + 1, 6))

In [191]:
intermediateParams[:,0] = np.arange(numMonths, -numMonths-1,-1)
intermediateParams[:,1] = np.arange(numMonths, -numMonths-1,-1)

In [192]:
for i in range(numMonths*2 + 1):
    intermediateParams[i,2] = intermediateParams[i,0] * np.exp(-a * deltat) - intermediateParams[i,1]

In [193]:
for i in range(numMonths*2 + 1):
    Beta = intermediateParams[i,2]
    intermediateParams[i,3] = 1/6 + 1/2*(Beta**2 + Beta)

In [194]:
for i in range(numMonths*2 + 1):
    Beta = intermediateParams[i,2]
    intermediateParams[i,4] = 2/3 - Beta**2

In [195]:
for i in range(numMonths*2 + 1):
    Beta = intermediateParams[i,2]
    intermediateParams[i,5] = 1/6 + 1/2*(Beta**2 - Beta)

## 1.3 ##

In [276]:
AD = np.zeros((numMonths*2 + 1,numMonths))
AD[numMonths,0] = 1

In [277]:
r = np.zeros((numMonths*2 + 1, numMonths))
r[numMonths,0] = -np.log(P(1))/(1/12)

In [278]:
from sympy import symbols, Eq, solve, lambdify

for i in range(1,numMonths):
    
    r0 = symbols('r0')
    P_sumAD = 0
    P_sumAD = -P(i) + AD[numMonths,i-1]*np.e**(-r0*deltat)*intermediateParams[numMonths,4] + AD[numMonths+1,i-1]*np.e**(-r0*deltat)*intermediateParams[numMonths,3] + AD[numMonths-1,i-1]*np.e**(-r0*deltat)*intermediateParams[numMonths,5]
    
    for j in range(i):
        
        Pd_down_iter = intermediateParams[numMonths + j,5]
        Pm_down_iter = intermediateParams[numMonths + j,4]
        Pu_down_iter = intermediateParams[numMonths + j,3]
        
        Pd_up_iter = intermediateParams[numMonths - j,5]
        Pm_up_iter = intermediateParams[numMonths - j,4]
        Pu_up_iter = intermediateParams[numMonths - j,3]
        
        #find one step forward Arrow-Debreu Prices
        AD[numMonths - j - 1, i ] = AD[numMonths - j , i - 1 ]*Pd_up_iter*np.exp(-r[numMonths - j, i - 1]*deltat) + AD[numMonths - j - 1, i - 1 ]*Pm_up_iter*np.exp(-r[numMonths - j - 1, i - 1]*deltat) + AD[numMonths - j - 2, i - 1 ]*Pu_up_iter*np.exp(-r[numMonths - j - 2, i - 1]*deltat)
        AD[numMonths, i ] = AD[numMonths + 1, i-1]*intermediateParams[numMonths,5]*np.exp(-r[numMonths + 1, i - 1]*deltat) + AD[numMonths, i-1]*intermediateParams[numMonths,4]*np.exp(-r[numMonths , i - 1]*deltat) + AD[numMonths +1, i-1]*intermediateParams[numMonths,3]*np.exp(-r[numMonths + 1, i - 1]*deltat)
        AD[numMonths + j + 1, i ] = AD[numMonths + j , i - 1 ]*Pd_down_iter*np.exp(-r[numMonths + j, i - 1]*deltat) + AD[numMonths + j + 1, i - 1 ]*Pm_down_iter*np.exp(-r[numMonths + j + 1, i - 1]*deltat) + AD[numMonths + j + 2, i - 1 ]*Pu_down_iter*np.exp(-r[numMonths + j + 2, i - 1]*deltat)   
        
    #update r values for one step ahead
    for j in range(i):
        # if i-j == 1
        if i-j == 1:
            P_sumAD += AD[numMonths-j,i-1]*np.e**(-r0*np.e**(OU[numMonths-j,i-1])*deltat)*intermediateParams[numMonths-j,3]
            P_sumAD += AD[numMonths+j,i-1]*np.e**(-r0*np.e**(OU[numMonths+j,i-1])*deltat)*intermediateParams[numMonths+j,5]

        # if i-j == 2
        if i-j == 2:
            P_sumAD += AD[numMonths-j,i-1]*np.e**(-r0*np.e**(OU[numMonths-j-1,i-1])*deltat)*intermediateParams[numMonths-j,3]
            P_sumAD += AD[numMonths+j,i-1]*np.e**(-r0*np.e**(OU[numMonths+j+1,i-1])*deltat)*intermediateParams[numMonths+j,5]

            P_sumAD += AD[numMonths-j-1,i-1]*np.e**(-r0*np.e**(OU[numMonths-j-1,i-1])*deltat)*intermediateParams[numMonths-j-1,4]
            P_sumAD += AD[numMonths+j+1,i-1]*np.e**(-r0*np.e**(OU[numMonths+j+1,i-1])*deltat)*intermediateParams[numMonths+j+1,4]

        # if i-j == 3
        if i-j == 3:
            P_sumAD += AD[numMonths-j,i-1]*np.e**(-r0*np.e**(OU[numMonths-j-1,i-1])*deltat)*intermediateParams[numMonths-j,3]
            P_sumAD += AD[numMonths+j,i-1]*np.e**(-r0*np.e**(OU[numMonths+j+1,i-1])*deltat)*intermediateParams[numMonths+j,5]

            P_sumAD += AD[numMonths-j-1,i-1]*np.e**(-r0*np.e**(OU[numMonths-j-1,i-1])*deltat)*intermediateParams[numMonths-j-1,4]
            P_sumAD += AD[numMonths+j+1,i-1]*np.e**(-r0*np.e**(OU[numMonths+j+1,i-1])*deltat)*intermediateParams[numMonths+j+1,4] 

            P_sumAD += AD[numMonths-j-2,i-1]*np.e**(-r0*np.e**(OU[numMonths-j-2,i-1])*deltat)*intermediateParams[numMonths-j-2,5]
            P_sumAD += AD[numMonths+j+2,i-1]*np.e**(-r0*np.e**(OU[numMonths+j+2,i-1])*deltat)*intermediateParams[numMonths+j+2,3]
      
    eq1 = Eq(P_sumAD)
    convert = lambdify('r0',P_sumAD)
    sol_r = convert(0)
    
    #update ri(0)
    r[numMonths, i] = sol_r                                              
    #update r matrix based off ri(0) * exp(OUij)
    for j in range(i):
        r[numMonths + j + 1 , i ] = r[numMonths, i] * np.exp(OU[numMonths + j + 1, i])
        r[numMonths - j - 1 , i ] = r[numMonths, i] * np.exp(OU[numMonths - j - 1, i])